In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from matplotlib import pyplot as plt
import numpy as np
import math

# Initialize Quantum Rings Simulator
provider = QuantumRingsProvider(token="rings-200.8l3J74L6HhxdoNjSDYxeypvZed3HCoIk", name="sanibir1977@gmail.com")
backend = provider.get_backend("scarlet_quantum_rings")
shots = 4096  # Increased number of shots

provider.active_account()

# Define core methods
def iqft_cct(qc, b, n):
    """
    Inverse Quantum Fourier Transform (IQFT) circuit.
    """
    for i in range(n):
        for j in range(1, i + 1):
            qc.cu1(-math.pi / 2 ** (i - j + 1), b[j - 1], b[i])
        qc.h(b[i])
    qc.barrier()
    return

def plot_histogram(counts, title=""):
    """
    Plots the histogram of measurement results.
    """
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    plt.bar(unique, bin_counts)
    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    plt.title(title)
    plt.show()
    return

def gcd(a, b):
    """
    Computes the greatest common divisor (GCD) of two numbers.
    """
    while b != 0:
        a, b = b, a % b
    return a

def shors_algorithm(N, a):
    # Check if a and N are coprime
    if math.gcd(a, N) != 1:
        print(f"{a} is not coprime with {N}. Trying another base...")
        return None, None

    # Quantum part: Find the period r
    n = math.ceil(math.log2(N))
    numberofqubits = 2 * n

    q = QuantumRegister(numberofqubits, 'q')
    c = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(q, c)

    # Apply Hadamard gates
    for i in range(n):
        qc.h(q[i])

    # Modular exponentiation (placeholder)
    for i in range(n):
        qc.cx(q[i], q[n + i])

    # Apply IQFT
    iqft_cct(qc, q, n)

    # Measure
    for i in range(n):
        qc.measure(q[i], c[i])

    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # Visualize results
    plot_histogram(counts, title=f"Shor's Algorithm for N={N}, a={a}")

    # Extract the period r
    measured_state = max(counts, key=counts.get)
    r = int(measured_state, 2)

    # Compute factors
    if r % 2 == 0:
        factor1 = math.gcd(a ** (r // 2) - 1, N)
        factor2 = math.gcd(a ** (r // 2) + 1, N)
        if factor1 != 1 and factor2 != 1:
            return factor1, factor2

    return None, None

# Test with N = 143
N = 143
a = 12  # Start with base a = 12
factor1, factor2 = shors_algorithm(N, a)
if factor1 and factor2:
    print(f"Factors of {N}: {factor1}, {factor2}")
else:
    print(f"Failed to factor {N} with base {a}. Trying another base...")
    a = 4  # Try another base
    factor1, factor2 = shors_algorithm(N, a)
    if factor1 and factor2:
        print(f"Factors of {N}: {factor1}, {factor2}")
    else:
        print(f"Failed to factor {N}.")